In [2]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
import splitfolders
import cv2
import numpy as np
import os
import matplotlib.pyplot as plt

In [ ]:
#Splitting the dataset
input_folder = 'Chaar Narayan\'
splitfolders.ratio(input_folder, output="Chaar Narayan", 
                   seed=42, ratio=(.7, .0, .3), 
                   group_prefix=None) 

In [1]:
dataset_path = r"Aayush\Chaar Narayan Temple\East"
def preprocess_data(image_path, label):
    img = cv2.imread(image_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (512, 512))
    
    datagen = ImageDataGenerator(
        rescale=1./255,
        rotation_range=20,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
    )
    
    params = datagen.get_random_transform(img.shape)
    augmented_img = datagen.apply_transform(img, params)
    augmented_img = np.clip(augmented_img, 0, 255).astype(np.uint8)
    
    plt.figure(figsize=(10, 5))
    plt.subplot(1, 2, 1)
    plt.imshow(img)
    plt.title("Original")
    plt.axis('off')

    plt.subplot(1, 2, 2)
    plt.imshow(augmented_img)
    plt.title("Augmentation")
    plt.axis('off')
    
    plt.show()
    
    images = [img, augmented_img]

    return [(image, label) for image in images]

In [16]:
cnn = tf.keras.models.Sequential()
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu', input_shape=[512, 512, 3]))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))
cnn.add(tf.keras.layers.Flatten())
cnn.add(tf.keras.layers.Dense(units=128, activation='relu'))
cnn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))
cnn.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [ ]:


datagen = ImageDataGenerator()
train_generator = datagen.flow(X_train, y_train, batch_size=32)

cnn.fit(train_generator, epochs=245, validation_data=(X_val, y_val))
